# Opracowanie modelu regresji do przewidzenia liczby pasażerów transportu kolejowego
#### Autorzy: Szymon Kowalski, Zuzanna Czyżowska - zespół 18
## 1. Wprowadzenie

W poprzedniej części projektu, przeprowadziliśmy analizę naszego zbioru i na jej podstawie dokonaliśmy doboru cech do naszego modelu. Cechy zapisaliśmy do nowego pliku .csv, aby łatwo przejść do opracowania modelu regresji.

## 2. Importy i załadowanie danych

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
tables = pd.read_csv('all_data_after_analyze.csv')

print("Dane:")
print(tables.head())
print("Rozmiar danych:")
print(tables.shape)

Dane:
    n        year  country     pa_tr  pa_tr_na  pa_tr_in     tr_tr  \
0  18  2013-01-01  Austria  235434.0  225063.0   10371.0  134386.0   
1  11  2013-01-01  Croatia   24216.0   23810.0     406.0   16651.0   
2   3  2013-01-01  Czechia  174189.0  170790.0    3399.0  127109.0   
3   4  2013-01-01  Denmark  193500.0  181151.0   12349.0   67045.0   
4   6  2013-01-01  Estonia    4199.0    4077.0     122.0    3337.0   

       le_ra_tr  le_li_hi  le_li_up_hi  ...  n_lo_di  n_ra_di  n_lo_el  ca_ve  \
0   9708.000000       0.0        237.0  ...    464.0    226.0    888.0  269.0   
1   4090.000000       0.0          0.0  ...    159.0    106.0    107.0   33.0   
2  15607.000000       0.0          0.0  ...   1126.0    737.0    770.0  288.0   
3   3613.330147       0.0          0.0  ...     89.0   1088.0     36.0  141.0   
4   2146.000000       0.0          0.0  ...    293.0     46.0      0.0   24.1   

     ex_in    in_in          po  co_pr_le   le_mo    n_pa_air  
0  319.000  211.000   

## 3. Modelowanie regresji
### 1. Przygotowanie zbioru treningowego i testowego

### 2. Ustawienie parametrów modelu

### 3. Trenowanie modelu

### 4. Wizualizacja wyników i ocena modelu

## 4. Regularyzacja modelu

### 1. Implementacja

### 2. Wizualizacja i ocena wyników oraz porównanie przed i po regularyzacji

## 5. Wnioski